In [2]:
# import secrets and tokens from config.py
from config import client_id, client_secret, refresh_token

import requests
import urllib3

# import packages for data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

In [3]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

Requesting Token...

Access Token = 9f4d924cd9a87564d20c8103a4a07b6b8bbae776



In [4]:
activities = pd.json_normalize(my_dataset)

In [6]:
activities.columns 

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'id', 'start_date',
       'start_date_local', 'timezone', 'utc_offset', 'location_city',
       'location_state', 'location_country', 'achievement_count',
       'kudos_count', 'comment_count', 'athlete_count', 'photo_count',
       'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged',
       'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed',
       'has_heartrate', 'average_heartrate', 'max_heartrate',
       'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high',
       'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score', 'athlete.id', 'athlete.resource_state', 'map.id',
       'map.summary_polyline', 'map.resource_state', 'workout_type',
       'average_cadence', 'average_watts', 'max_watts',
       'weighted_a

In [5]:
#Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date
activities.head(5)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,...,map.resource_state,workout_type,average_cadence,average_watts,max_watts,weighted_average_watts,kilojoules,device_watts,average_temp,start_time
0,2,Afternoon Swim,1629.7,1765,2455,0.0,Swim,Swim,9610042593,2023-08-08T23:45:01Z,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16:45:01
1,2,Evening HIIT,0.0,3584,3584,0.0,Workout,HighIntensityIntervalTraining,9603649228,2023-08-08T02:02:21Z,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19:02:21
2,2,Morning Run,5648.0,1878,1902,4.0,Run,Run,9601083751,2023-08-07T16:39:54Z,...,2,NaN,81.7,302.3,401.0,299.0,567.8,True,NaN,09:39:54
3,2,NFS McKinley Magic Miles,5041.3,2140,2148,7.0,Run,Run,9594866140,2023-08-06T15:04:57Z,...,2,0.0,80.1,238.6,409.0,238.0,510.7,True,NaN,08:04:57
4,2,Morning Run,13229.5,4146,4215,18.0,Run,Run,9588446455,2023-08-05T14:20:36Z,...,2,0.0,83.5,318.5,520.0,319.0,1320.4,True,NaN,07:20:36


In [8]:
# Make a copy of activities DataFrame for testing.
activities_copy = activities.copy()

In [12]:
activities_copy.drop(
    [
        'map.summary_polyline', 
        'resource_state',
        'external_id', 
        'upload_id', 
        'location_city', 
        'location_state', 
        'has_kudoed', 
        'start_date', 
        'athlete.resource_state', 
        'utc_offset', 
        'map.resource_state', 
        'athlete.id', 
        'visibility', 
        'heartrate_opt_out', 
        'upload_id_str', 
        'from_accepted_tag', 
        'map.id', 
        'manual', 
        'private', 
        'flagged', 
    ], 
    axis=1, 
    inplace=True
)

In [13]:
# filter down to Ride, Run, and Swim activities
activities_copy = activities_copy.query("type == 'Run'")
print(len(activities_copy))

111


In [17]:
# convert data types

# what does this do?
# activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)
# what does this do?
activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)
# convert values
activities_copy.loc[:, 'distance'] /= 1609.344 # convert from meters to miles
activities_copy.loc[:, 'average_speed'] *= 2.23693629 # convert from meters/second to miles/hour
activities_copy.loc[:, 'max_speed'] *= 2.23693629 # convert from meters/second to miles/hour
# set index
#activities_copy.set_index('start_date_local', inplace=True)

C:\Users\jluong\AppData\Local\Temp\ipykernel_12476\4159416693.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)


In [ ]:
# rename columns

In [18]:
# https://stackoverflow.com/questions/25146121/extracting-just-month-and-year-separately-from-pandas-datetime-column
# create a column that extracts month and year from the activity
# df['yyyy-mm'] = pd.to_datetime(df['ArrivalDate']).dt.strftime('%Y-%m')
activities_copy['Month_Year'] = pd.to_datetime(activities_copy['start_date_local']).dt.strftime('%Y-%m')

In [ ]:
# https://docs.python.org/3/library/datetime.html#datetime.date.isocalendar
# make a Week column


In [ ]:
# group runs by week

# get weekly mileage

# get total weekly moving time

# calculate average speed for the week

In [ ]:
# set weekly mileage targets


In [ ]:
# plot weekly mileage